### **初始化**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/[2023-24] Internship Preparation/西南证券/9. 固收+基金/数据

In [ ]:
import pandas as pd
import numpy as np
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

def deal_with_nan(data): # 处理缺失值
    df = data.copy()
    nanpos = df.isna().astype(int)
    loc = nanpos.diff().replace(np.nan,0)
    for i in range(data.shape[1]):
        try:
            locpre = np.where(loc.iloc[:,i]==-1)[0][0]
            backfill = df.iloc[:,[i]].bfill()
            interpolate = df.iloc[:,[i]].interpolate()
            df.iloc[:,[i]] = pd.concat([backfill.iloc[:locpre],interpolate.iloc[locpre:]])
        except:
            continue
    return df

def ratio_sa(data): # 季节性调整：比例因子法
    df = data.copy()
    for i in range(data.shape[1]):
        df.iloc[:,i] = df.iloc[:,i].groupby(df.iloc[:,i].index.month).apply(lambda x: x/x.mean())
    return df

def multiplicative_sa(data): # 季节性调整：乘法模型
    multp = statsmodels.tsa.seasonal.seasonal_decompose(data,model='multiplicative')
    T = multp.trend; S = multp.seasonal; R = multp.resid
    return data/S # remove seasonality

def laggedcorr_lead(data, bchmk, direction=1): # 时差相关系数 + 领先期数
    idx = list(set(data.index).intersection(set(bchmk.index))) # 调整时期
    lagcoef = [np.corrcoef(data.loc[idx].shift(-i).dropna(), bchmk.loc[idx].iloc[:-i].dropna())[0][1] for i in range(1,13)] # 只考虑领先期数为1到12时的相关系数
    lead = np.argmax(lagcoef) if direction == 1 else np.argmin(lagcoef)
    return data.name, lagcoef[lead], lead+1

def get_kl_lead(data,bchmk,direction=1): # K-L信息量领先月数
    def get_kl(lead):
        p = bchmk.div(bchmk.sum()) # 基准
        q = data.div(data.sum())*direction # 指标
        idx = sorted(list(set(data.index).intersection(set(bchmk.index))))
        klinfo = (p.loc[idx].iloc[:-lead]*np.log(p.loc[idx].iloc[:-lead]*q.loc[idx].shift(-lead).dropna()).ffill()).dropna().replace([np.inf,-np.inf],np.nan).ffill().sum()*10000 # 只考虑领先期数
        return klinfo
    kls = [get_kl(i) for i in range(1,13)] # 领先期数取值范围为1到12
    return data.name, np.argmin(kls)+1

start = '2010-12-31'; end = '2023-12-31'

### **研报复现**

In [ ]:
### 参考文章：https://mp.weixin.qq.com/s?__biz=MzA4MDk3NjY0Mw==&mid=2652850772&idx=1&sn=14d52ef7f4f834b6905cb625eb5604e3&chksm=8477e438b3006d2efbc7823dca30837b88677f8b41cfef52b4a0830668d5b346c16b630f71db&scene=27#wechat_redirect
############### 宏观指标
### 数据
data = pd.read_excel('宏观代理指标.xlsx').dropna(how='all').iloc[:-2].rename(columns={'指标名称':'日期'}).set_index('日期').resample('M').last().loc[start:end]
data.index = pd.to_datetime(data.index)
stock = data.loc[:,['中国:产量:发电量:当月值','中国:产量:铝材:当月值','中国:产量:硫酸:当月值','中国:产量:乙烯:当月值','中国:产量:空调:当月值','中国:产量:汽车:当月值',
                    '中国:销量:叉车:全行业:当月值','中国:房屋新开工面积:累计值','中国:房地产开发投资完成额:累计值','中国:货物周转量:当月值','中国:税收收入:当月值']] # 总量类指标, 增长代理
stock.loc[:,['中国:房屋新开工面积:累计值','中国:房地产开发投资完成额:累计值']] = stock.loc[:,['中国:房屋新开工面积:累计值','中国:房地产开发投资完成额:累计值']].diff() # 累计值转当月值
stock = deal_with_nan(stock)
price = deal_with_nan(data.loc[:,['中国:大宗价:猪肉','CRB现货指数:油脂','期货结算价(连续):布伦特原油','中国:钢材价格指数:螺纹钢(Φ16)']]) # 价格类指标, 通胀代理

### 春节效应/季节性调整
"""
# x13: https://www.statsmodels.org/dev/generated/statsmodels.tsa.x13.x13_arima_analysis.html
statsmodels.tsa.x13.x13_arima_analysis(price,x12path=1)
# 比例因子法
stockSA = ratio_sa(stock)
priceSA = ratio_sa(price)
"""
# 乘法模型: https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html / https://blog.51cto.com/u_16175524/9864554 / https://otexts.com/fppcn/components.html
stockSA = pd.concat([multiplicative_sa(stock.iloc[:,i]) for i in range(stock.shape[1])],axis=1).rename(columns=dict(zip(range(stock.shape[1]),stock.columns.tolist())))
priceSA = pd.concat([multiplicative_sa(price.iloc[:,i]) for i in range(price.shape[1])],axis=1).rename(columns=dict(zip(range(price.shape[1]),price.columns.tolist())))

### 预处理 & 同比增速
stockC = pd.concat([sm.tsa.filters.hpfilter(stockSA.iloc[:,i],lamb=14400)[0] for i in range(stock.shape[1])],axis=1).pct_change(12).dropna()
priceT = pd.concat([sm.tsa.filters.hpfilter(priceSA.iloc[:,i],lamb=1)[1] for i in range(price.shape[1])],axis=1).pct_change(12).dropna()


############### 领先指标筛选筛选
### 基准
benchmark = pd.read_excel('基准.xlsx').iloc[:-2].rename(columns={'指标名称':'日期'}).set_index('日期').resample('M').last() # unit: %
benchmark.iloc[:,0] = benchmark.iloc[:,0].pct_change(12)*100 # PMI转同比
benchmark = benchmark.dropna().reindex(stockC.index)
### 时差相关系数
# 增长因子基准为同比口径的PMI
# 通胀因子基准为同比口径的CPI和PPI
lagcor = pd.concat([pd.DataFrame([list(laggedcorr_lead(stockC.iloc[:,i], benchmark.iloc[:,0])) for i in range(stockC.shape[1])]).rename(columns=dict(zip(range(3),['指标名称','时差相关系数','时差相关系数领先月数']))),
                    pd.DataFrame([list(laggedcorr_lead(priceT.iloc[:,i], benchmark.iloc[:,1])) for i in range(2)]).rename(columns=dict(zip(range(3),['指标名称','时差相关系数','时差相关系数领先月数']))),
                    pd.DataFrame([list(laggedcorr_lead(priceT.iloc[:,i], benchmark.iloc[:,2])) for i in range(2,priceT.shape[1])]).rename(columns=dict(zip(range(3),['指标名称','时差相关系数','时差相关系数领先月数'])))]).reset_index(drop=True)
### K-L信息量
kl = pd.concat([pd.DataFrame([list(get_kl_lead(stockC.iloc[:,i], benchmark.iloc[:,0])) for i in range(stockC.shape[1])]).rename(columns=dict(zip(range(2),['指标名称','K-L信息量领先月数']))),
                pd.DataFrame([list(get_kl_lead(priceT.iloc[:,i], benchmark.iloc[:,0])) for i in range(2)]).rename(columns=dict(zip(range(2),['指标名称','K-L信息量领先月数']))),
                pd.DataFrame([list(get_kl_lead(priceT.iloc[:,i], benchmark.iloc[:,0])) for i in range(2,priceT.shape[1])]).rename(columns=dict(zip(range(2),['指标名称','K-L信息量领先月数'])))]).reset_index(drop=True)
### 拐点匹配率

### DTW距离
